In [1]:
import pandas as pd

In [23]:
df = pd.read_csv("./data/SMSSpamCollection", sep="\t", names=["classification", "messages"])
df

,classification,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
df.columns

Index(['classification', 'messages'], dtype='object')

In [4]:
df["classification"].unique()

array(['ham', 'spam'], dtype=object)

In [24]:
df["spam"] = (df["classification"] == "spam").astype(int)
df = df.drop("classification", axis=1)

In [25]:
df.tail(2)

,messages,spam
5570,The guy did some bitching but I acted like i'd...,0
5571,Rofl. Its true to its name,0


In [26]:
df["spam"]

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: spam, Length: 5572, dtype: int64

In [8]:
(~df["spam"]).sum()

np.int64(4825)

In [68]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


X_train_text, X_test_text, y_train_series, y_test_series = train_test_split(
    df["messages"], df["spam"], test_size=0.2, random_state=42, stratify=df["spam"]
)

In [55]:
import numpy as np

In [69]:
print((np.sum(y_train_series== 0))/ (np.sum(y_train_series==1)))
print((np.sum(y_test_series==0))/(np.sum(y_test_series==1)))

# stratify makes sure that both the train and test set have the same proportions of spam and ham messages :)


6.453177257525083
6.483221476510067


In [57]:
count = CountVectorizer(max_features=1000, stop_words="english")


In [58]:
X_train_count = count.fit_transform(X_train_text)
X_test_count = count.transform(X_test_text)

In [59]:
import torch
from torch import nn

In [74]:
# convering to pytorch tensorsss
X_train = torch.tensor(X_train_count.toarray(), dtype=torch.float32)
X_test = torch.tensor(X_test_count.toarray(), dtype=torch.float32)

y_train = torch.tensor(y_train_series.values, dtype=torch.float32).reshape(-1, 1)
y_test = torch.tensor(y_test_series.values, dtype=torch.float32).reshape(-1, 1)

In [76]:
print(X_train.shape, y_train.shape)

torch.Size([4457, 1000]) torch.Size([4457, 1])


In [82]:
class SpamClassifier(nn.Module):
    def __init__(self, *, input_size):
        super(SpamClassifier, self).__init__()
        self.classifier = nn.Sequential(
            
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1)
        )
        
    def forward(self,x):
        return self.classifier(x)

            

In [84]:
model = SpamClassifier(input_size=X_train.shape[1])
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay=1e-5)

In [92]:
def train_net(model, X_train, X_test, y_train, y_test, epochs=100):
    train_losses = []
    test_accuracies = []
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        train_outputs = model(X_train)
        train_loss = loss_fn(train_outputs, y_train)
        train_loss.backward()
        optimizer.step()
        
        
        if epoch % 10 == 0 :  # evaluating only once per ten steps
            
            model.eval()  # this ignores the dropouttt
            with torch.no_grad():
                test_output = model(X_test)
                test_prob = torch.sigmoid(test_output)
                test_preds = (test_prob >= 0.5).float()
                test_accuracy = (test_preds == y_test).float().mean()
                
                train_losses.append(train_loss.item())
                test_accuracies.append(test_accuracy.item())
                
                print(f'Epoch {epoch}: Train Loss = {train_loss.item():.4f}, Test Accuracy = {test_accuracy.item():.4f}')

    return train_losses, test_accuracies

                

In [93]:
train_losses, test_accuracies = train_net(model, X_train,X_test,y_train, y_test, epochs=200)


Epoch 0: Train Loss = 0.7126, Test Accuracy = 0.1336
Epoch 10: Train Loss = 0.6783, Test Accuracy = 0.9040
Epoch 20: Train Loss = 0.6098, Test Accuracy = 0.9812
Epoch 30: Train Loss = 0.4734, Test Accuracy = 0.9794
Epoch 40: Train Loss = 0.3049, Test Accuracy = 0.9803
Epoch 50: Train Loss = 0.1735, Test Accuracy = 0.9812
Epoch 60: Train Loss = 0.1042, Test Accuracy = 0.9821
Epoch 70: Train Loss = 0.0668, Test Accuracy = 0.9830
Epoch 80: Train Loss = 0.0460, Test Accuracy = 0.9848
Epoch 90: Train Loss = 0.0323, Test Accuracy = 0.9848
Epoch 100: Train Loss = 0.0253, Test Accuracy = 0.9830
Epoch 110: Train Loss = 0.0205, Test Accuracy = 0.9830
Epoch 120: Train Loss = 0.0171, Test Accuracy = 0.9821
Epoch 130: Train Loss = 0.0142, Test Accuracy = 0.9812
Epoch 140: Train Loss = 0.0126, Test Accuracy = 0.9821
Epoch 150: Train Loss = 0.0102, Test Accuracy = 0.9821
Epoch 160: Train Loss = 0.0097, Test Accuracy = 0.9821
Epoch 170: Train Loss = 0.0086, Test Accuracy = 0.9821
Epoch 180: Train Loss

In [97]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [98]:
# final evaluation:
model.eval()  
with torch.no_grad():
    train_outputs = model(X_train)
    train_probs = torch.sigmoid(train_outputs)
    train_preds = (train_probs >= 0.5).float()
    train_accuracy = (train_preds == y_train).float().mean()
    
    test_outputs = model(X_test)
    test_probs = torch.sigmoid(test_outputs)
    test_preds = (test_probs >= 0.5).float()
    test_accuracy = (test_preds == y_test).float().mean()
    
    print(f"\nFinal Results:")
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    y_test_np = y_test.numpy().flatten()
    test_preds_np = test_preds.numpy().flatten()
    
    print("\nClassification Report:")
    print(classification_report(y_test_np, test_preds_np, target_names=['Ham', 'Spam']))
    
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test_np, test_preds_np))



Final Results:
Training Accuracy: 0.9996
Test Accuracy: 0.9803

Classification Report:
              precision    recall  f1-score   support

         Ham       0.98      1.00      0.99       966
        Spam       0.97      0.88      0.92       149

    accuracy                           0.98      1115
   macro avg       0.98      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115


Confusion Matrix:
[[962   4]
 [ 18 131]]


In [99]:
# meh result but still okay i suppose

In [101]:
def predict_message(message, model, vectorizer, threshold=0.5):
    """Predict if a message is spam or ham"""
    model.eval()
    
    
    message_vector = vectorizer.transform([message])
    message_tensor = torch.tensor(message_vector.toarray(), dtype=torch.float32)
    
    
    with torch.no_grad():
        logits = model(message_tensor)
        probability = torch.sigmoid(logits).item()
        prediction = "spam" if probability >= threshold else "ham"
    
    return prediction, probability


print("\n" + "="*50)
print("SPAM CLASSIFIER - TEST YOUR MESSAGES")
print("="*50)

test_messages = [
    "Hello, how are you doing today?",
    "FREE! Click here to win $1000 now! Limited time offer!",
    "Can you pick me up from the airport at 3pm?",
    "URGENT: Your account will be closed. Click here immediately!",
    "Thanks for dinner last night, it was great!"
]

print("Testing with example messages:")
for msg in test_messages:
    pred, prob = predict_message(msg, model, count)
    print(f"Message: '{msg[:50]}...' -> {pred.upper()} (confidence: {prob:.3f})")

# Interactive mode
print("\nNow test your own messages (type 'quit' to exit):")
while True:
    user_message = input("\nEnter a message: ")
    if user_message.lower() == 'quit':
        break
    
    try:
        prediction, probability = predict_message(user_message, model, count)
        confidence = max(probability, 1 - probability)
        print(f"Prediction: {prediction.upper()}")
        print(f"Confidence: {confidence:.3f}")
        print(f"Spam probability: {probability:.3f}")
    except Exception as e:
        print(f"Error processing message: {e}")

print("Thanks for using the spam classifier!")


SPAM CLASSIFIER - TEST YOUR MESSAGES
Testing with example messages:
Message: 'Hello, how are you doing today?...' -> HAM (confidence: 0.002)
Message: 'FREE! Click here to win $1000 now! Limited time of...' -> SPAM (confidence: 0.520)
Message: 'Can you pick me up from the airport at 3pm?...' -> HAM (confidence: 0.008)
Message: 'URGENT: Your account will be closed. Click here im...' -> SPAM (confidence: 0.842)
Message: 'Thanks for dinner last night, it was great!...' -> HAM (confidence: 0.000)

Now test your own messages (type 'quit' to exit):
Prediction: HAM
Confidence: 1.000
Spam probability: 0.000
Prediction: HAM
Confidence: 0.511
Spam probability: 0.489
Prediction: HAM
Confidence: 0.990
Spam probability: 0.010
Prediction: SPAM
Confidence: 0.757
Spam probability: 0.757
Thanks for using the spam classifier!
